# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""../../../../da…","""21299364521099…","""10663593066270…",1.2927e19,"""98427614285609…","""13352465235166…",false
"""../../../../da…","""18402887482138…","""24723785218702…",1.4856e19,"""98427614285609…","""31258234234857…",true
"""../../../../da…","""15415537547478…","""12834114050594…",5.6458e17,"""14144817760672…","""13352465235166…",false
"""../../../../da…","""18402887482138…","""15008317121162…",4.0723e18,"""18066536610648…","""13352465235166…",false
"""../../../../da…","""21299364521099…","""92904430923903…",1.4677e19,"""18066536610648…","""10211729355007…",true
"""../../../../da…","""15415537547478…","""62701079443247…",1.6330e19,"""76970820996358…",null,false
"""../../../../da…","""15415537547478…","""15814842802250…",8.7675e18,"""18066536610648…","""10211729355007…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""12330604238011…","""14757800479140…",1.2927e19,"""FirstMortgage3…","""91044029019124…","""Rejected"""
"""../../../../da…","""26511972791204…","""89282673054583…",1.4856e19,"""FirstMortgage3…","""19173353074300…","""Accepted"""
"""../../../../da…","""38620538188546…","""18053901497030…",5.6458e17,"""MoneyMarketSav…","""91044029019124…","""Rejected"""
"""../../../../da…","""26511972791204…","""10933703277702…",4.0723e18,"""BasicChecking""","""91044029019124…","""Rejected"""
"""../../../../da…","""12330604238011…","""76753222573203…",1.4677e19,"""BasicChecking""","""56773351394436…","""Accepted"""
"""../../../../da…","""38620538188546…","""56287046312406…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""38620538188546…","""15000966808313…",8.7675e18,"""BasicChecking""","""56773351394436…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""11404547384918…","""43062171125591…",1.2927e19,"""FirstMortgage3…","""94572400789038…","""Rejected"""
"""../../../../da…","""17181087556677…","""53690163446344…",1.4856e19,"""FirstMortgage3…","""31515402810507…","""Accepted"""
"""../../../../da…","""18438105527449…","""35113049559343…",5.6458e17,"""MoneyMarketSav…","""94572400789038…","""Rejected"""
"""../../../../da…","""17181087556677…","""12469279897069…",4.0723e18,"""BasicChecking""","""94572400789038…","""Rejected"""
"""../../../../da…","""11404547384918…","""72029287291846…",1.4677e19,"""BasicChecking""","""11025177936841…","""Accepted"""
"""../../../../da…","""18438105527449…","""66779901994881…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""18438105527449…","""17172116703971…",8.7675e18,"""BasicChecking""","""11025177936841…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_MaritalStatus=PREDICTOR_1
Customer_City=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""17504707935294…","""15845156209276…",1.2927e19,"""FirstMortgage3…","""10149009894560…","""Rejected"""
"""53873372893109…","""13860477619638…",1.4856e19,"""FirstMortgage3…","""29249144800946…","""Accepted"""
"""65357475894913…","""12534100378957…",5.6458e17,"""MoneyMarketSav…","""10149009894560…","""Rejected"""
"""53873372893109…","""12418375478126…",4.0723e18,"""BasicChecking""","""10149009894560…","""Rejected"""
"""17504707935294…","""59071463433610…",1.4677e19,"""BasicChecking""","""93337275302481…","""Accepted"""
"""65357475894913…","""14836065567992…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""65357475894913…","""17618505657411…",8.7675e18,"""BasicChecking""","""93337275302481…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'